# Signal Processing Guideline


In [ ]:
import os
import numpy as np
import quantities as pq
import matplotlib.pyplot as plt



## 1. Data Load

In [ ]:
from miv.io import load_data

In [ ]:
# Load dataset from OpenEphys recording
folder_path: str = "~/Open Ephys/2022-03-10-16-19-09"  # Data Path
dataset = load_data(folder_path, device="OpenEphys")

### 1.1. Meta Data Structure

In [ ]:
# Get signal and rate(hz)
record_node: int = dataset.get_nodes[0]
recording = dataset[record_node]["experiment1"]["recording1"]
signal, _, rate = recording.continuous["100"]
# time = recording.continuous["100"].timestamp / rate
num_channels = signal.shape[1]

### 1.2. Array Data Structure

### 1.3 Raw Data

## 2. Filtering Raw Signal

We provide a set of basic signal filter tools. It is highly recommended to filter the signal before doing the spike-detection.
Here, we provide examples of how to create and apply the filter to the [`dataset`](../api/io.rst).

If you have further suggestion on other filters to include, please leave an issue on our [GitHub issue page](https://github.com/GazzolaLab/MiV-OS/issues) with `enhancement` tag.

In [ ]:
from miv.signal.filter import FilterCollection, ButterBandpass

### 2.1 Filter Collection

[Here](../api/signal.html#filter) is the list of provided filters.
All filters are `Callable`, taking `signal` and `sampling_rate` as parameters.
To define a multiple filters together, we provide [`FilterCollection`](../api/_toctree/FilterAPI/miv.signal.filter.FilterCollection) that execute multiple filters in a series.


In [ ]:
# Butter bandpass filter
pre_filter = ButterBandpass(lowcut=300, highcut=3000, order=5)

# How to construct sequence of filters
pre_filter = (
    FilterCollection(tag="Filter Example")
        .append(ButterBandpass(lowcut=300, highcut=3000, order=5))
        #.append(Limiter(400*pq.mV))
        #.append(Filter1(**filter1_kwargs))
        #.append(Filter2(**filter2_kwargs))
)

### 2.2 Apply Filter

There are two way to apply the filter on the signal.
- If the signal is stored in `numpy array` format, you can directly call the filter `prefilter(signal, sampling_rate)`.
- If you want to apply the filter to all signals in the `dataset`, `dataset` provide `.apply_filter` method that takes any `filter` (any filter that abide [`filter protocol`](../api/_toctree/FilterAPI/miv.signal.filter.FilterProtocol)).
  - You can select [subset of `dataset`](../api/dataset.html#data-subset) and [mask-out channels](../api/dataset.html#mask-channel) before applying the filter.
  
You can check the list of all provided filters [here](../api/signal.html#filter).

In [ ]:
# Apply filter to entire dataset
dataset.apply_filter(pre_filter)
filtered_signal = dataset[record_node]['experiment1']['recording1'].filtered_signal

# Apply filter to array
rate = 30_000
filtered_signal = pre_filter(data_array, sampling_rate=rate)

# Retrieve data from dataset and apply filter
data = dataset[record_node]['experiment1']['recording1']
filtered_signal = pre_filter(data, sampling_rate=rate)

## 3. Spike Detection

You can check the available method [here](../api/signal.html#spike-detection).

Most simple example of spike-detection method is using `ThresholdCutoff`.

In [ ]:
from miv.signal.spike import ThresholdCutoff

In [ ]:
# Define spike-detection method
spike_detection = ThresholdCutoff()

# The detection can be used directly as the following.
#   signal        : np.array or neo.core.AnalogSignal, shape(N_channels, N)
#   timestamps    : np.array, shape(N) 
#   sampling_rate : float
timestamps = spike_detection(signal, timestamps, sampling_rate=30_000, cutoff=3.5)

# The detection can be applied on the dataset
dataset.apply_spike_detection(spike_detection)

## 4. Spike Visualization

In [ ]:
import neo
from viziphant.rasterplot import rasterplot_rates

In [ ]:
# Plot
rasterplot_rates(spiketrain_list)